Here we apply the Rearing classification algorithum that was discussed to a specific case study

In [0]:
import random
import numpy as np
from PIL import Image
import math
import pandas as pd 
from sklearn import svm
import os
from sklearn.metrics import zero_one_loss
from sklearn.metrics import confusion_matrix

In [0]:
Marker_predictions_1=pd.read_csv('/content/drive/My Drive/test8_data_senior_design/test8/Test8PART1DLC_resnet50_Trial4Mar23shuffle1_10000.csv',header=1)
Marker_predictions_2=pd.read_csv('/content/drive/My Drive/test8_data_senior_design/test8/Test8PART2DLC_resnet50_Trial4Mar23shuffle1_10000.csv',header=1)
Marker_predictions_3=pd.read_csv('/content/drive/My Drive/test8_data_senior_design/test8/Test8PART3DLC_resnet50_Trial4Mar23shuffle1_10000.csv',header=1)
Marker_predictions_4=pd.read_csv('/content/drive/My Drive/test8_data_senior_design/test8/Test8PART4DLC_resnet50_Trial4Mar23shuffle1_10000.csv',header=1)
Marker_predictions_5=pd.read_csv('/content/drive/My Drive/test8_data_senior_design/test8/Test8PART5DLC_resnet50_Trial4Mar23shuffle1_10000.csv',header=1)

rearing_gt=pd.read_excel('/content/drive/My Drive/test8_data_senior_design/test8/trial8_rearing_complete.xlsx',header=0)

# organize rearing data # frame rate of 15
rearing_gt_1=rearing_gt[5400:22500].to_numpy() # start at sixth minute, there are 19 minutes in each segment therefore 17100 frames in between 
rearing_gt_2=rearing_gt[27900:45000].to_numpy()
rearing_gt_3=rearing_gt[50400:67500].to_numpy()
rearing_gt_4=rearing_gt[72900:90000].to_numpy()
rearing_gt_5=rearing_gt[95400:112500].to_numpy()
data_rearing_labels=np.concatenate((rearing_gt_1[:,1],rearing_gt_2[:,1],rearing_gt_3[:,1],rearing_gt_4[:,1],rearing_gt_5[:,1]))

#organizing marker predictions
Marker_predictions_1=Marker_predictions_1.iloc[825:17925] # start at 55 seconds after and end at the 20th exactly
Marker_predictions_2=Marker_predictions_2.iloc[825:17925]
Marker_predictions_3=Marker_predictions_3.iloc[825:17925]
Marker_predictions_4=Marker_predictions_4.iloc[825:17925]
Marker_predictions_5=Marker_predictions_5.iloc[825:17925]
Marker_predictions = [Marker_predictions_1, Marker_predictions_2, Marker_predictions_3, Marker_predictions_4, Marker_predictions_5]
Marker_predictions = pd.concat(Marker_predictions)


snout_xy_train=Marker_predictions[["snout","snout.1"]].to_numpy().astype(float)
centroid_xy_train=Marker_predictions[["rightear","rightear.1"]].to_numpy().astype(float)
tailbase_xy_train=Marker_predictions[["tailbase","tailbase.1"]].to_numpy().astype(float)
hip_right_xy_train=Marker_predictions[["righthip","righthip.1"]].to_numpy().astype(float)
hip_left_xy_train=Marker_predictions[["lefthip","lefthip.1"]].to_numpy().astype(float)

def Euclidean_distance(x_1,y_1,x_2,y_2):
    euc_dist=np.sqrt((((x_1-x_2)**2) + ((y_1-y_2)**2)))
    return euc_dist


#features
distance_snout_tailbase=Euclidean_distance(snout_xy_train[:,0],snout_xy_train[:,1],tailbase_xy_train[:,0],tailbase_xy_train[:,1])
distance_snout_right_hip=Euclidean_distance(snout_xy_train[:,0],snout_xy_train[:,1],hip_right_xy_train[:,0],hip_right_xy_train[:,1])
distance_snout_left_hip=Euclidean_distance(snout_xy_train[:,0],snout_xy_train[:,1],hip_left_xy_train[:,0],hip_left_xy_train[:,1])
prob_centroid=Marker_predictions[["centroid.2","rightcentroid.2","leftcentroid.2"]].to_numpy().astype(float)

features=np.column_stack((distance_snout_tailbase,distance_snout_right_hip,distance_snout_left_hip,prob_centroid))



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Train the SVM model on the controll data below

In [0]:
clf = svm.SVC(kernel='linear')
clf.fit(features[0:17100,:],data_rearing_labels[0:17100])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
predictions_2=clf.predict(features[17100:17100*2,:])
loss=zero_one_loss(data_rearing_labels[17100:17100*2], predictions_2) 
Accuracy=1-loss
print('Overall Testing Accuracy: {}'.format(Accuracy))

predictions_3=clf.predict(features[17100*2:17100*3])
loss=zero_one_loss(data_rearing_labels[17100*2:17100*3], predictions_3) 
Accuracy=1-loss
print('Overall Testing Accuracy: {}'.format(Accuracy))

predictions_4=clf.predict(features[17100*3:17100*4])
loss=zero_one_loss(data_rearing_labels[17100*3:17100*4], predictions_4) 
Accuracy=1-loss
print('Overall Testing Accuracy: {}'.format(Accuracy))

predictions_5=clf.predict(features[17100*4:17100*5])
loss=zero_one_loss(data_rearing_labels[17100*4:17100*5], predictions_5) 
Accuracy=1-loss
print('Overall Testing Accuracy: {}'.format(Accuracy))


Overall Testing Accuracy: 0.9546198830409357
Overall Testing Accuracy: 0.9749122807017544
Overall Testing Accuracy: 0.9869005847953216
Overall Testing Accuracy: 0.7988888888888889


We consider Prevelence of Rearing to be the percentage of frames where our SVM predicts the model to be rearing


In [0]:
Prevelence_2=((sum(predictions_2))/len(predictions_2))
Prevelence_3=((sum(predictions_3))/len(predictions_3))
Prevelence_4=((sum(predictions_4))/len(predictions_4))
Prevelence_5=((sum(predictions_5))/len(predictions_5))

print(Prevelence_2)
print(Prevelence_3)
print(Prevelence_4)
print(Prevelence_5)

0.02064327485380117
0.010058479532163742
0.0032748538011695906
0.12461988304093567
